In [1]:
import  tensorflow as tf
from    tensorflow.keras import datasets
import numpy as np

In [2]:
import os
tf.random.set_seed(22)
np.random.seed(22)

In [3]:
# 设定最常用的
total_words = 10000
(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words=total_words)

In [4]:
(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(
                                                path = './imdb.npz',
                                                num_words = total_words)

In [5]:
print(x_train.shape,  y_train.shape)
print(x_test.shape, y_test.shape)

(25000,) (25000,)
(25000,) (25000,)


In [6]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [12]:
y_train[0]

1

In [13]:
len(x_train[0])

218

In [14]:
word_index = datasets.imdb.get_word_index()

1646592/1641221 [==============================] - 1s 1us/step


In [ ]:
for word, code in word_index.items():
    print(word,code)

In [16]:
word_index = {word :(code + 3) for word, code in word_index.items()}
word_index["<PAD>"] = 0 # 填充标志
word_index["<START>"] = 1 # 起始标志
word_index["<UNK>"] = 2 # 未知单词的标志
word_index["<UNUSED>"] = 3

In [17]:
# 翻转编码表
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [18]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [19]:
decode_review(x_train[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [8]:
from    tensorflow import keras
max_review_len = 80
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)

In [9]:

batchsz = 128
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.shuffle(1000).batch(batchsz, drop_remainder=True)
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.batch(batchsz, drop_remainder=True)
print('x_train shape:', x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
x_test shape: (25000, 80)


In [24]:
print('x_train shape:', x_train.shape)

x_train shape: (25000, 80)


In [25]:
print('x_test shape:', x_test.shape)

x_test shape: (25000, 80)


In [28]:
###简易RNN
class MyRNN(keras.Model):

    def __init__(self, units):
        super(MyRNN, self).__init__()


        # transform text to embedding representation
        # [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len,
                                          input_length=max_review_len)

        # [b, 80, 100] , h_dim: 64
        self.rnn = keras.Sequential([
            layers.SimpleRNN(units, dropout=0.5, return_sequences=True, unroll=True),
            layers.SimpleRNN(units, dropout=0.5, unroll=True)
        ])


        # fc, [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = layers.Dense(1)

    def call(self, inputs, training : bool = True):
        """
        net(x) net(x, training=True) :train mode
        net(x, training=False): test
        :param inputs: [b, 80]
        :param training:
        :return:
        """
        # [b, 80]
        x = inputs
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute
        # x: [b, 80, 100] => [b, 64]
        x = self.rnn(x)

        # out: [b, 64] => [b, 1]
        x = self.outlayer(x)
        # p(y is pos|x)
        prob = tf.sigmoid(x)

        return prob

In [26]:
#LSTM

embedding_len = 100
from    tensorflow.keras import layers, losses, optimizers, Sequential

class MyRNN(keras.Model):

    def __init__(self, units):
        super(MyRNN, self).__init__()


        # transform text to embedding representation
        # [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len,
                                          input_length=max_review_len)

        # [b, 80, 100] , h_dim: 64
        self.rnn = keras.Sequential([
            # layers.SimpleRNN(units, dropout=0.5, return_sequences=True, unroll=True),
            # layers.SimpleRNN(units, dropout=0.5, unroll=True)

            layers.LSTM(units, dropout=0.5, return_sequences=True, unroll=True),
            layers.LSTM(units, dropout=0.5, unroll=True)
        ])


        # fc, [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = layers.Dense(1)

    def call(self, inputs, training : bool = True):
        """
        net(x) net(x, training=True) :train mode
        net(x, training=False): test
        :param inputs: [b, 80]
        :param training:
        :return:
        """
        # [b, 80]
        x = inputs
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute
        # x: [b, 80, 100] => [b, 64]
        x = self.rnn(x)

        # out: [b, 64] => [b, 1]
        x = self.outlayer(x)
        # p(y is pos|x)
        prob = tf.sigmoid(x)

        return prob



In [30]:
#GRU
class MyRNN(keras.Model):

    def __init__(self, units):
        super(MyRNN, self).__init__()


        # transform text to embedding representation
        # [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len,
                                          input_length=max_review_len)

        # [b, 80, 100] , h_dim: 64
        self.rnn = keras.Sequential([
            # layers.SimpleRNN(units, dropout=0.5, return_sequences=True, unroll=True),
            # layers.SimpleRNN(units, dropout=0.5, unroll=True)

            # unroll: Boolean (default False). If True, the network will be unrolled,
            # else a symbolic loop will be used.
            # Unrolling can speed-up a RNN, although it tends to be more memory-intensive.
            # Unrolling is only suitable for short sequences.
            layers.GRU(units, dropout=0.5, return_sequences=True, unroll=True),
            layers.GRU(units, dropout=0.5, unroll=True)
        ])


        # fc, [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = layers.Dense(1)

    def call(self, inputs, training : bool = True):
        """
        net(x) net(x, training=True) :train mode
        net(x, training=False): test
        :param inputs: [b, 80]
        :param training:
        :return:
        """
        # [b, 80]
        x = inputs
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute
        # x: [b, 80, 100] => [b, 64]
        x = self.rnn(x)

        # out: [b, 64] => [b, 1]
        x = self.outlayer(x)
        # p(y is pos|x)
        prob = tf.sigmoid(x)

        return prob

In [31]:
def main():
    units = 64 # RNN状态向量长度f
    epochs = 10 # 训练epochs

    training = True
    
    model = MyRNN(units)

    if training:
        # 装配
        model.compile(optimizer = optimizers.RMSprop(0.001),
                      loss = losses.BinaryCrossentropy(),
                      metrics=['accuracy'],
                      experimental_run_tf_function = False)
    else:
        # 装配
        model.compile(optimizer = optimizers.RMSprop(0.001),
          loss = losses.BinaryCrossentropy(),
          metrics=['accuracy'])

   

    # 训练和验证
    model.fit(db_train, epochs=epochs, validation_data=db_test)
    # 测试
    model.evaluate(db_test)


if __name__ == '__main__':
    main()


Epoch 1/10
195/195 [==============================] - 51s 260ms/step - loss: 0.5104 - accuracy: 0.6554 - val_loss: 0.3562 - val_accuracy: 0.8405
Epoch 2/10
195/195 [==============================] - 19s 95ms/step - loss: 0.3497 - accuracy: 0.8449 - val_loss: 0.3637 - val_accuracy: 0.8446
Epoch 3/10
195/195 [==============================] - 19s 96ms/step - loss: 0.3064 - accuracy: 0.8679 - val_loss: 0.3564 - val_accuracy: 0.8447
Epoch 4/10
195/195 [==============================] - 18s 92ms/step - loss: 0.2824 - accuracy: 0.8851 - val_loss: 0.3566 - val_accuracy: 0.8440
Epoch 5/10
195/195 [==============================] - 18s 90ms/step - loss: 0.2618 - accuracy: 0.8956 - val_loss: 0.3510 - val_accuracy: 0.8465
Epoch 6/10
195/195 [==============================] - 17s 89ms/step - loss: 0.2434 - accuracy: 0.9035 - val_loss: 0.3644 - val_accuracy: 0.8450
Epoch 7/10
195/195 [==============================] - 18s 91ms/step - loss: 0.2283 - accuracy: 0.9102 - val_loss: 0.4242 - val_accuracy